# The Battle of Neighborhoods (Week 2)

## Introduction - Business Problem

The objective is to find a proper location and provide some recommendation to install an office/start-up specifically in **Lisbon,  Portugal**. Considering the current situation and all  uncertainties related to the Covid-19, Portugal is the <a href="https://startupportugal.com/whyportugal">third safest country in the world </a>offering stability in troubled times. Additionally, Portugal provides incentives for startups, investors, and R&D.

Based on which district Lisbon is pretty diversified and offers various facilities, so let’s try to **identify available facilities** in this city preferably close to the city center.
The categories of venues that matter to employees of this start-up have been identified: Yoga Studio, Café, Coworking Spaces.


## Data

Based on the business problem described above, our focus will be:

* What are the existing Coworking Spaces, Coffee Places and Yoga Studio in the neighborhood
* Distance to coworking spaces and café in the neighborhood, if any
* Distance of neighborhood from city center

Regarding the data sources that will be used:
* Using **Foursquare API** to get the number of Coworking Spaces, Coffee Places and Yoga space and location in every neighborhood
* Coordinate of Lisbon city center will be obtained using Foursquare API of well known Lisbon location (Praça do Comercio)


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported


Let's take define Foursquare Credentials and Version. As a version, to get some conclusive results, a date prior to Covid-19 has been chosen.

In [2]:
CLIENT_ID = 'VH412NZW0DUNVUU1TYUQUKUPBR3TRJN13PGN3XQP23WT4IBP'
CLIENT_SECRET = '0EJOYEPEVBNHBWPSJITOCSZ4HZVREGAJ430GUFUZKOIH3ZOC'
VERSION = '20191201'
LIMIT = 30

Let's define the coordinates of one of main place in the city center next to the river (Praça do Comércio). 

In [3]:
address = 'Praça do Comércio, 1100-148 Lisboa'

geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.707555 -9.136283


Let's look at a specific venue category 'Coworking' within 5 km from Praça do Comércio.

In [4]:
search_query = 'Coworking'
radius = 5000
print(search_query + '.....OK!')

Coworking.....OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VH412NZW0DUNVUU1TYUQUKUPBR3TRJN13PGN3XQP23WT4IBP&client_secret=0EJOYEPEVBNHBWPSJITOCSZ4HZVREGAJ430GUFUZKOIH3ZOC&ll=38.707555,-9.136283&v=20191201&query=Coworking&radius=5000&limit=30'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed4cabd1a4b0a001b669920'},
 'response': {'venues': [{'id': '50ce25d119a9a6e2e1ed10fe',
    'name': 'Avila Coworking-Lisboa',
    'location': {'address': 'Av República 6 7 Dir',
     'lat': 38.734820363276015,
     'lng': -9.14485216140747,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.734820363276015,
       'lng': -9.14485216140747}],
     'distance': 3125,
     'postalCode': '1050-191',
     'cc': 'PT',
     'city': 'Lisboa',
     'state': 'Lisboa',
     'country': 'Portugal',
     'formattedAddress': ['Av República 6 7 Dir',
      '1050-191 Lisboa',
      'Portugal']},
    'categories': [{'id': '4bf58dd8d48988d174941735',
      'name': 'Coworking Space',
      'pluralName': 'Coworking Spaces',
      'shortName': 'Coworking Space',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/office_coworkingspace_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1591003679',
    'hasPerk': Fal

We assign the relevant part of JSON to venues and tranform venues into a **dataframe**

In [7]:
venues = results ['response'] ['venues']

dataframe = json_normalize(venues)
dataframe.head()

/Users/emmanuelleshaaravi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,50ce25d119a9a6e2e1ed10fe,Avila Coworking-Lisboa,"[{'id': '4bf58dd8d48988d174941735', 'name': 'C...",v-1591003679,False,Av República 6 7 Dir,38.734820,-9.144852,"[{'label': 'display', 'lat': 38.73482036327601...",3125,1050-191,PT,Lisboa,Lisboa,Portugal,"[Av República 6 7 Dir, 1050-191 Lisboa, Portugal]"
1,547354b4498e3f6064eabc33,#CoworkingEU - North America HQ,"[{'id': '4bf58dd8d48988d174941735', 'name': 'C...",v-1591003679,False,NaN,38.722485,-9.135495,"[{'label': 'display', 'lat': 38.72248458862305...",1663,NaN,PT,NaN,NaN,Portugal,[Portugal]
2,596e4039a4b51b482ad641b8,Work-o,"[{'id': '4bf58dd8d48988d174941735', 'name': 'C...",v-1591003679,False,R. Maria Andrade 8,38.723158,-9.132688,"[{'label': 'display', 'lat': 38.723158, 'lng':...",1764,1170-214,PT,Lisboa,Lisboa,Portugal,"[R. Maria Andrade 8, 1170-214 Lisboa, Portugal]"
3,5820b4b142f0d55756f76717,as oficinas Co-working Space,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1591003679,False,Rua do Benformoso 225,38.719927,-9.135191,"[{'label': 'display', 'lat': 38.719927, 'lng':...",1380,1100,PT,Lisboa,Lisboa,Portugal,"[Rua do Benformoso 225, 1100 Lisboa, Portugal]"


Let's refine the information, analyse it and visualize.

In [8]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = ['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress
0,Avila Coworking-Lisboa,Coworking Space,Av República 6 7 Dir,38.734820,-9.144852,"[{'label': 'display', 'lat': 38.73482036327601...",3125,1050-191,PT,Lisboa,Lisboa,Portugal,"[Av República 6 7 Dir, 1050-191 Lisboa, Portugal]"
1,#CoworkingEU - North America HQ,Coworking Space,NaN,38.722485,-9.135495,"[{'label': 'display', 'lat': 38.72248458862305...",1663,NaN,PT,NaN,NaN,Portugal,[Portugal]
2,Work-o,Coworking Space,R. Maria Andrade 8,38.723158,-9.132688,"[{'label': 'display', 'lat': 38.723158, 'lng':...",1764,1170-214,PT,Lisboa,Lisboa,Portugal,"[R. Maria Andrade 8, 1170-214 Lisboa, Portugal]"
3,as oficinas Co-working Space,Office,Rua do Benformoso 225,38.719927,-9.135191,"[{'label': 'display', 'lat': 38.719927, 'lng':...",1380,1100,PT,Lisboa,Lisboa,Portugal,"[Rua do Benformoso 225, 1100 Lisboa, Portugal]"


In [9]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)# generate map centered around Praça do Comércio

# add a red circle marker to represent Praça do Comércio
folium.features.CircleMarker(
    [latitude,longitude],
    radius = 10,
    color='red',
    popup='Praça do Comércio',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6,
).add_to(venues_map)

# add the Coworking Spaces as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

We can see that around Intendente, some coworking spaces have already been implemented. This could mean to be a good location. 

In [45]:
address = 'Intendente, Lisbon'

geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.7226276 -9.1353016


So now let's have a look at the category 'Café' within 1 km from Intendente.

In [46]:
search_query = 'Cafe'
radius = 1000
print(search_query + '.....OK!')

Cafe.....OK!


In [47]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VH412NZW0DUNVUU1TYUQUKUPBR3TRJN13PGN3XQP23WT4IBP&client_secret=0EJOYEPEVBNHBWPSJITOCSZ4HZVREGAJ430GUFUZKOIH3ZOC&ll=38.7226276,-9.1353016&v=20191201&query=Cafe&radius=1000&limit=30'

In [48]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed4da3c216785001b557a92'},
 'response': {'venues': [{'id': '4b0588a1f964a520d8d022e3',
    'name': 'Hard Rock Cafe Lisboa',
    'location': {'address': 'Avenida Da Liberdade, 2',
     'crossStreet': 'R. dos Condes',
     'lat': 38.716656798227234,
     'lng': -9.14176681844856,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.716656798227234,
       'lng': -9.14176681844856}],
     'distance': 870,
     'postalCode': '1250-144',
     'cc': 'PT',
     'city': 'Lisboa',
     'state': 'Lisboa',
     'country': 'Portugal',
     'formattedAddress': ['Avenida Da Liberdade, 2 (R. dos Condes)',
      '1250-144 Lisboa',
      'Portugal']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referra

In [49]:
venues = results ['response'] ['venues']

dataframe = json_normalize(venues)
dataframe.head()

/Users/emmanuelleshaaravi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4b0588a1f964a520d8d022e3,Hard Rock Cafe Lisboa,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1591007836,False,"Avenida Da Liberdade, 2",R. dos Condes,38.716657,-9.141767,"[{'label': 'display', 'lat': 38.71665679822723...",870,1250-144,PT,Lisboa,Lisboa,Portugal,"[Avenida Da Liberdade, 2 (R. dos Condes), 1250...",NaN,NaN
1,55a27400498ed70fbc55d3f6,Giradiscos Cafe,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1591007836,False,"Travessa do Monte, 30",NaN,38.718534,-9.130606,"[{'label': 'display', 'lat': 38.718534, 'lng':...",611,1170-108,PT,Lisboa,Lisboa,Portugal,"[Travessa do Monte, 30, 1170-108 Lisboa, Portu...",NaN,NaN
2,4d481a12847e6dcb11229bc4,Mundo do Café Delta,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1591007836,False,C. C. Tivoli Forum,"Av. da Liberdade, 180",38.720165,-9.144986,"[{'label': 'display', 'lat': 38.72016544078836...",884,1250-146,PT,Lisboa,Lisboa,Portugal,"[C. C. Tivoli Forum (Av. da Liberdade, 180), 1...",NaN,NaN
3,58fcbe40b3d8e242db406020,Everest Bhet Ghat Cafe,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591007836,False,NaN,NaN,38.722430,-9.135190,"[{'label': 'display', 'lat': 38.72243, 'lng': ...",24,NaN,PT,Lisboa,Lisboa,Portugal,"[Lisboa, Portugal]",NaN,NaN
4,4e6aa88845ddadf2d093be24,Vox Café,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1591007836,False,"R. da Voz do Operário, 13",NaN,38.715422,-9.128977,"[{'label': 'display', 'lat': 38.7154223619129,...",972,NaN,PT,Lisboa,1100,Portugal,"[R. da Voz do Operário, 13, Lisboa, Portugal]",NaN,NaN


Let's refine the information, analyse it and visualize.

In [50]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = ['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood
0,Hard Rock Cafe Lisboa,American Restaurant,"Avenida Da Liberdade, 2",R. dos Condes,38.716657,-9.141767,"[{'label': 'display', 'lat': 38.71665679822723...",870,1250-144,PT,Lisboa,Lisboa,Portugal,"[Avenida Da Liberdade, 2 (R. dos Condes), 1250...",NaN
1,Giradiscos Cafe,Café,"Travessa do Monte, 30",NaN,38.718534,-9.130606,"[{'label': 'display', 'lat': 38.718534, 'lng':...",611,1170-108,PT,Lisboa,Lisboa,Portugal,"[Travessa do Monte, 30, 1170-108 Lisboa, Portu...",NaN
2,Mundo do Café Delta,Café,C. C. Tivoli Forum,"Av. da Liberdade, 180",38.720165,-9.144986,"[{'label': 'display', 'lat': 38.72016544078836...",884,1250-146,PT,Lisboa,Lisboa,Portugal,"[C. C. Tivoli Forum (Av. da Liberdade, 180), 1...",NaN
3,Everest Bhet Ghat Cafe,Restaurant,NaN,NaN,38.722430,-9.135190,"[{'label': 'display', 'lat': 38.72243, 'lng': ...",24,NaN,PT,Lisboa,Lisboa,Portugal,"[Lisboa, Portugal]",NaN
4,Vox Café,Bar,"R. da Voz do Operário, 13",NaN,38.715422,-9.128977,"[{'label': 'display', 'lat': 38.7154223619129,...",972,NaN,PT,Lisboa,1100,Portugal,"[R. da Voz do Operário, 13, Lisboa, Portugal]",NaN
5,belississimo cafe,Portuguese Restaurant,NaN,NaN,38.712552,-9.132366,"[{'label': 'display', 'lat': 38.71255166049879...",1150,NaN,PT,NaN,NaN,Portugal,[Portugal],NaN
6,Café Buenos Aires,Argentinian Restaurant,"Calçada do Duque, 31 B",NaN,38.713471,-9.141719,"[{'label': 'display', 'lat': 38.71347068779724...",1161,1100,PT,Lisboa,Lisboa,Portugal,"[Calçada do Duque, 31 B, 1100 Lisboa, Portugal]",NaN
7,Cyber Café-Bar,Bar,Rua da Atalaia 153,NaN,38.713187,-9.145147,"[{'label': 'display', 'lat': 38.71318661, 'lng...",1354,NaN,PT,Lisboa,Lisboa,Portugal,"[Rua da Atalaia 153, Lisboa, Portugal]",NaN
8,Asas De Café,Chinese Restaurant,R. Andrade 28,NaN,38.722646,-9.134405,"[{'label': 'display', 'lat': 38.722646, 'lng':...",77,1170-016,PT,Lisboa,Lisboa,Portugal,"[R. Andrade 28, 1170-016 Lisboa, Portugal]",NaN
9,Café O Bicas,Café,NaN,NaN,38.723400,-9.133256,"[{'label': 'display', 'lat': 38.7234001159668,...",197,NaN,PT,NaN,NaN,Portugal,[Portugal],NaN


In [51]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)# generate map centred around Intendente

# add a red circle marker to represent Intendente
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [latitude,longitude],
        radius = 10,
        color='red',
        popup='Intendente',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.6,
    ).add_to(venues_map)

# add the Coffee Places as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color='yellow',
        popup=label,
        fill=True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

So we can see that various coffee place exist in that area which is good! But we also need to take into account that the Start-up founder really enjoy doing some yoga classes during working hours. So let's now have a look at the yoga places within a reasonable distance (1.5 km). Therefore, we decided to keep the area of Intendente as a point of reference.

In [68]:
search_query2 = 'Yoga'
radius = 1500
print(search_query2 + '.....OK!')

Yoga.....OK!


In [69]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VH412NZW0DUNVUU1TYUQUKUPBR3TRJN13PGN3XQP23WT4IBP&client_secret=0EJOYEPEVBNHBWPSJITOCSZ4HZVREGAJ430GUFUZKOIH3ZOC&ll=38.7226276,-9.1353016&v=20191201&query=Yoga&radius=1500&limit=30'

In [70]:
results2 = requests.get(url).json()
results2

{'meta': {'code': 200, 'requestId': '5ed4de335fb726001b2a578f'},
 'response': {'venues': [{'id': '5228c43711d28632a039c239',
    'name': 'Academia Yoga Darshan Portugal',
    'location': {'address': 'R. António Pedro, 1B',
     'lat': 38.72846317457428,
     'lng': -9.135651771038718,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.72846317457428,
       'lng': -9.135651771038718}],
     'distance': 650,
     'postalCode': '1150-044',
     'cc': 'PT',
     'city': 'Lisboa',
     'state': 'Lisboa',
     'country': 'Portugal',
     'formattedAddress': ['R. António Pedro, 1B',
      '1150-044 Lisboa',
      'Portugal']},
    'categories': [{'id': '4bf58dd8d48988d175941735',
      'name': 'Gym / Fitness Center',
      'pluralName': 'Gyms or Fitness Centers',
      'shortName': 'Gym / Fitness',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1591008619',
    'hasPerk': Fals

In [74]:
venues2 = results2 ['response'] ['venues']

dataframe2 = json_normalize(venues2)
dataframe2.head()

/Users/emmanuelleshaaravi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,5228c43711d28632a039c239,Academia Yoga Darshan Portugal,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1591008619,False,"R. António Pedro, 1B",38.728463,-9.135652,"[{'label': 'display', 'lat': 38.72846317457428...",650,1150-044,PT,Lisboa,Lisboa,Portugal,"[R. António Pedro, 1B, 1150-044 Lisboa, Portugal]",NaN
1,4d9760609aec59410bb57164,Yoga Áshrama Rato,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",v-1591008619,False,"Largo do Rato, nº4",38.728399,-9.148061,"[{'label': 'display', 'lat': 38.72839889723553...",1280,1250 186,PT,Lisboa,Lisboa,Portugal,"[Largo do Rato, nº4, 1250 186 Lisboa, Portugal]",NaN
2,4dfdc5e518a8eee4fe41647b,Yoga darshan,"[{'id': '4bf58dd8d48988d1b2941735', 'name': 'C...",v-1591008619,False,NaN,38.728024,-9.135275,"[{'label': 'display', 'lat': 38.728024, 'lng':...",600,NaN,PT,Lisboa,Lisboa,Portugal,"[Lisboa, Portugal]",NaN
3,5736fef3498eeae082e2d8b4,Centro António Pereira Yoga e Lifestyle,"[{'id': '4bf58dd8d48988d171941735', 'name': 'E...",v-1591008619,False,NaN,38.728982,-9.137575,"[{'label': 'display', 'lat': 38.728982, 'lng':...",734,NaN,PT,NaN,NaN,Portugal,[Portugal],NaN
4,55b3e8e8498e9b5f30b34064,Yoga Live Academy,"[{'id': '4bf58dd8d48988d131941735', 'name': 'S...",v-1591008619,False,Rua Antônio Pedro 1,38.731313,-9.134473,"[{'label': 'display', 'lat': 38.731313, 'lng':...",969,NaN,PT,NaN,NaN,Portugal,"[Rua Antônio Pedro 1, Portugal]",NaN


Let's refine the information, analyse it and visualize.

In [75]:
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')]
dataframe2_filtered = dataframe2.loc[:, filtered_columns2]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = ['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe2_filtered['categories'] = dataframe2_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe2_filtered.columns = [column.split('.')[-1] for column in dataframe2_filtered.columns]

dataframe2_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet
0,Academia Yoga Darshan Portugal,Gym / Fitness Center,"R. António Pedro, 1B",38.728463,-9.135652,"[{'label': 'display', 'lat': 38.72846317457428...",650,1150-044,PT,Lisboa,Lisboa,Portugal,"[R. António Pedro, 1B, 1150-044 Lisboa, Portugal]",NaN
1,Yoga Áshrama Rato,Yoga Studio,"Largo do Rato, nº4",38.728399,-9.148061,"[{'label': 'display', 'lat': 38.72839889723553...",1280,1250 186,PT,Lisboa,Lisboa,Portugal,"[Largo do Rato, nº4, 1250 186 Lisboa, Portugal]",NaN
2,Yoga darshan,College Gym,NaN,38.728024,-9.135275,"[{'label': 'display', 'lat': 38.728024, 'lng':...",600,NaN,PT,Lisboa,Lisboa,Portugal,"[Lisboa, Portugal]",NaN
3,Centro António Pereira Yoga e Lifestyle,Event Space,NaN,38.728982,-9.137575,"[{'label': 'display', 'lat': 38.728982, 'lng':...",734,NaN,PT,NaN,NaN,Portugal,[Portugal],NaN
4,Yoga Live Academy,Spiritual Center,Rua Antônio Pedro 1,38.731313,-9.134473,"[{'label': 'display', 'lat': 38.731313, 'lng':...",969,NaN,PT,NaN,NaN,Portugal,"[Rua Antônio Pedro 1, Portugal]",NaN
5,Ministry of Yoga,Yoga Studio,"R. das Portas de Santo Antão 27, Floor 1",38.715318,-9.139741,"[{'label': 'display', 'lat': 38.7153178579982,...",900,1150-264,PT,Lisboa,Lisboa,Portugal,"[R. das Portas de Santo Antão 27, Floor 1, 115...",NaN
6,Little Yoga Space,Spiritual Center,"Rua da Assunção 42, 1. floor, room 3",38.711796,-9.137448,"[{'label': 'display', 'lat': 38.711796, 'lng':...",1220,1100-044,PT,NaN,Lisboa,Portugal,"[Rua da Assunção 42, 1. floor, room 3, 1100-04...",NaN
7,Padma Yoga,Yoga Studio,NaN,38.734379,-9.143207,"[{'label': 'display', 'lat': 38.73437881469726...",1477,NaN,PT,NaN,NaN,Portugal,[Portugal],NaN
8,"Espaço Vida - Yoga, Psicologia e Bem-estar",Yoga Studio,"Avenida Guerra Junqueiro, 11",38.738318,-9.136115,"[{'label': 'display', 'lat': 38.738318, 'lng':...",1748,1000-166,PT,Lisboa,Lisboa,Portugal,"[Avenida Guerra Junqueiro, 11, 1000-166 Lisboa...",NaN
9,Holmes Place Defensores de Chaves,Gym / Fitness Center,"Av. Defensores de Chaves, 45-B",38.736906,-9.143956,"[{'label': 'display', 'lat': 38.73690604385417...",1758,1000-112,PT,Lisboa,Lisboa,Portugal,"[Av. Defensores de Chaves, 45-B (Av. António J...",Av. António José de Almeida


In [76]:
venues_map2 = folium.Map(location=[latitude, longitude], zoom_start=13)# generate map centred around Intendente

# add a red circle marker to represent Intendente
for lat, lng, label in zip(dataframe2_filtered.lat, dataframe2_filtered.lng, dataframe2_filtered.categories):
    folium.features.CircleMarker(
        [latitude,longitude],
        radius = 10,
        color='red',
        popup='Intendente',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.6,
    ).add_to(venues_map2)

# add the Coffee Places as blue circle markers
for lat, lng, label in zip(dataframe2_filtered.lat, dataframe2_filtered.lng, dataframe2_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color='fuchsia',
        popup=label,
        fill=True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map2)

venues_map2